References

1. [Whisper](https://blog.devgenius.io/transcribing-youtube-videos-using-openais-whisper-%EF%B8%8F-%EF%B8%8F-a29d264d6fb1)
2. [Langchain and LLama](https://www.youtube.com/watch?v=k_1pOF1mj8k)
3. [English to Hindi using Transformers](https://prateekjoshi.medium.com/english-to-hindi-translation-made-simple-with-transformers-library-33f64f745552)

In [1]:
from IPython.display import YouTubeVideo
YouTubeVideo('https://www.youtube.com/watch?v=CuBzyh4Xmvk', width=500, height=300)

### Basic Imports

In [2]:
try:
    import yt_dlp
except ImportError:
    %pip install yt_dlp
    import yt_dlp

In [2]:
def download(video_id: str, save_path: str) -> str:
    video_url = f'https://www.youtube.com/watch?v={video_id}'
    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'paths': {'home': save_path},
        'outtmpl': {'default': "lecture.m4a"},
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'm4a',
        }]
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([video_url])
        if error_code != 0:
            raise Exception('Failed to download video')

    return save_path

In [3]:
download('CuBzyh4Xmvk', '../datasets/audio/')

[youtube] Extracting URL: https://www.youtube.com/watch?v=CuBzyh4Xmvk
[youtube] CuBzyh4Xmvk: Downloading webpage
[youtube] CuBzyh4Xmvk: Downloading ios player API JSON
[youtube] CuBzyh4Xmvk: Downloading android player API JSON
[youtube] CuBzyh4Xmvk: Downloading m3u8 information
[info] CuBzyh4Xmvk: Downloading 1 format(s): 140
[download] ../datasets/audio/lecture.m4a has already been downloaded
[download] 100% of   72.26MiB
[ExtractAudio] Not converting audio ../datasets/audio/lecture.m4a; file is already in target format m4a


'../datasets/audio/'

In [4]:
try:
    from pydub import AudioSegment
except ImportError:
    %pip install pydub
    %pip install pydub[extras]
    from pydub import AudioSegment
    from pydub.playback import play


In [5]:
from IPython.display import Audio
audio_path = '../datasets/audio/lecture.m4a'
audio = AudioSegment.from_file(audio_path, format="m4a")


In [6]:
audio[:13000]

In [4]:
try:    
    import whisper
except ImportError:
    %pip install openai-whisper
    import whisper

In [5]:
whisper_model = whisper.load_model("base.en")


In [6]:
transcription = whisper_model.transcribe("../datasets/audio/lecture.m4a", fp16=True, verbose=False)

100%|██████████| 468481/468481 [02:05<00:00, 3734.80frames/s]


In [11]:
print(transcription["text"][:500].replace(". ", "\n"))

 Please look at the code mentioned above and please sign up on the Google Cloud
We've already started making some announcements
You will likely end up missing the announcements and you'll have no one else to play with
The second quick logistical announcement is that we'll have an extra lecture on Saturday, 11th Jan at 11am in 1.101
So a lot of ones over there
And I think one or two people still have conflict, but in the larger, in the larger phone we will have almost everyone available, so 


In [12]:
transcription.keys()

dict_keys(['text', 'segments', 'language'])

In [13]:
def create_srt_from_transcription(transcription_objects, srt_file_path):
    with open(srt_file_path, 'w') as srt_file:
        index = 1  # SRT format starts with index 1

        for entry in transcription_objects['segments']:
            start_time = entry['start']
            end_time = entry['end']
            text = entry['text']

            # Convert time to SRT format
            start_time_str = format_time(start_time)
            end_time_str = format_time(end_time)

            # Write entry to SRT file
            srt_file.write(f"{index}\n")
            srt_file.write(f"{start_time_str} --> {end_time_str}\n")
            srt_file.write(f"{text}\n\n")

            index += 1

def format_time(time_seconds):
    minutes, seconds = divmod(time_seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},000"


In [14]:
create_srt_from_transcription(transcription, "../datasets/audio/lecture.srt")

In [15]:
!head ../datasets/audio/lecture.srt

1
00:00:00,000 --> 00:00:05,000
 Please look at the code mentioned above and please sign up on the Google Cloud.

2
00:00:05,000 --> 00:00:08,000
 We've already started making some announcements.

3
00:00:08,000 --> 00:00:14,000


In [16]:
try:
    from gtts import gTTS
except ImportError:
    %pip install gtts
    from gtts import gTTS

Note: you may need to restart the kernel to use updated packages.


In [17]:
tts = gTTS(transcription["text"][:1300], lang='en')

In [18]:
with open('../datasets/audio/hello.mp3', 'wb') as f:
    tts.write_to_fp(f)

In [20]:
from IPython.display import Audio
Audio('../datasets/audio/hello.mp3')

In [21]:
%pip install transformers -U -q
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 1.3 MB 1.8 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [22]:
try:
    from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
except:
    %pip install transformers -U -q
    %pip install sentencepiece
    from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
    

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [23]:

# download and save model
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

# import tokenizer
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

In [27]:
text_to_translate = transcription["text"][:500].split(". ")
text_to_translate

[' Please look at the code mentioned above and please sign up on the Google Cloud',
 "We've already started making some announcements",
 "You will likely end up missing the announcements and you'll have no one else to play with",
 "The second quick logistical announcement is that we'll have an extra lecture on Saturday, 11th Jan at 11am in 1.101",
 'So a lot of ones over there',
 'And I think one or two people still have conflict, but in the larger, in the larger phone we will have almost everyone available, so ']

In [28]:
model_inputs = tokenizer(text_to_translate, return_tensors="pt", padding=True, truncation=True)

In [29]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
)

translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)



['कृपया उपर्युक्त कोड को देखें और कृपया Google क्लाउड पर साइन अप करें', 'हम पहले से ही कुछ घोषणाएं करने शुरू कर दी हैं', 'आप शायद अंत में घोषणाओं को खो देंगे और आप के साथ खेलने के लिए कोई अन्य नहीं होगा', 'दूसरा त्वरित लॉजिस्टिक घोषणा यह है कि हम एक अतिरिक्त व्याख्यान Saturday, 11th Jan 11am में 1.101 में होगा', 'तो वहाँ के बहुत से', 'और मुझे लगता है कि एक या दो लोग अभी भी संघर्ष है, लेकिन बड़ी, बड़ी फोन में हम लगभग सभी उपलब्ध हो जाएगा, तो']


In [30]:
translation

['कृपया उपर्युक्त कोड को देखें और कृपया Google क्लाउड पर साइन अप करें',
 'हम पहले से ही कुछ घोषणाएं करने शुरू कर दी हैं',
 'आप शायद अंत में घोषणाओं को खो देंगे और आप के साथ खेलने के लिए कोई अन्य नहीं होगा',
 'दूसरा त्वरित लॉजिस्टिक घोषणा यह है कि हम एक अतिरिक्त व्याख्यान Saturday, 11th Jan 11am में 1.101 में होगा',
 'तो वहाँ के बहुत से',
 'और मुझे लगता है कि एक या दो लोग अभी भी संघर्ष है, लेकिन बड़ी, बड़ी फोन में हम लगभग सभी उपलब्ध हो जाएगा, तो']

In [31]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 
                                 

In [32]:
llm = Ollama(model="mistral", 
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [36]:
prompt_qs = ["Please provide a bullet-point summary for the given text:",
             "Highlight the important topics and subtopics in the given lecture:",
             "Give us some question for a quiz based on the following text:",
             "Summarize the following text in Hindi:",
             "Summarize the following text in Hindi transliteration:",]

prompts = [q + "\n\n" + transcription["text"] for q in prompt_qs]

for prompt, prompt_qs in zip(prompts, prompt_qs):
    print(prompt_qs, end="\n\n")
    output = llm(prompt)
    print(output, end="\n\n")
    print("=="*50, end="\n\n")

Please provide a bullet-point summary for the given text:

 * The text mentions signing up on Google Cloud and upcoming announcements
* There will be an extra lecture on Saturday, 11th Jan at 11am in room 1.101
* The code and course website have been put on Google Cloud
* Recap of previous discussion: machine learning is a method for computers to learn without explicit programming by analyzing patterns from data
* Discussion on recognizing digits using machine learning with examples and rules
* Overview of difference between traditional programming and machine learning, with emphasis on the importance of experience and data in machine learning
* Introduction to a new problem: predicting quality or condition for computer vision using images as input
* Explanation of how machine learning can be used to scale processes that require human input
* Discussion of types of machine learning tasks: classification (discrete output variables) and regression (continuous output variables)
* Introduc

KeyboardInterrupt: 